In [9]:
!pip install diffusers
!pip install nltk
import nltk
nltk.download('punkt')
import nltk
nltk.download('averaged_perceptron_tagger')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
!pip install streamlit


In [11]:
%%writefile app.py
import streamlit as st
import torch
from diffusers import SemanticStableDiffusionPipeline
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def text_to_prompts(text):
    # Split the text into sentences using full stops as delimiters
    sentences = text.split(".")

    # Remove any empty strings resulting from consecutive full stops
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    return sentences

def find_adjective_noun_in_prompts(prompts):
    all_adjective_noun_pairs = []
    for prompt in prompts:
        tokens = word_tokenize(prompt)
        tagged_words = pos_tag(tokens)

        adjective_noun_pairs = []
        for i in range(len(tagged_words) - 1):
            word, tag = tagged_words[i]
            next_word, next_tag = tagged_words[i + 1]
            if tag.startswith('JJ') and next_tag.startswith('NN'):
                adjective_noun_pairs.append((word, next_word))

        all_adjective_noun_pairs.extend(adjective_noun_pairs)
    return all_adjective_noun_pairs

def main():
    st.title("Image Generation from Text")

    # Text input box
    text_input = st.text_area("Enter the text paragraph:")

    if st.button("Generate Images"):
        # Load the Diffusion Model pipeline
        pipe = SemanticStableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to(device='cuda')

        # Define the prompts
        prompts = text_to_prompts(text_input)

        # Find adjective-noun pairs in prompts
        all_adjective_noun_pairs = find_adjective_noun_in_prompts(prompts)

        # Construct editing prompts using adjective-noun pairs
        editing_prompts = [", ".join(pair) for pair in all_adjective_noun_pairs]

        # Set up the generator
        gen = torch.Generator(device='cuda')
        gen.manual_seed(21)

        # Generate images based on the prompts and editing prompts
        images = []
        for prompt, editing_prompt in zip(prompts, editing_prompts):
            try:
                out = pipe(
                    prompt=prompt,
                    generator=gen,
                    num_images_per_prompt=1,
                    guidance_scale=7,
                    editing_prompt=[editing_prompt],
                    reverse_editing_direction=[False],
                    edit_warmup_steps=[10],
                    edit_guidance_scale=[4],
                    edit_threshold=[0.99],
                    edit_momentum_scale=0.3,
                    edit_mom_beta=0.6,
                    edit_weights=[1]
                )
                images.extend(out.images)
            except Exception as e:
                st.error(f"Error generating images: {e}")

        # Display all generated images
        num_images = len(images)
        st.text(f"Number of generated images: {num_images}")

        for i, image in enumerate(images):
            st.image(image, caption=f"Generated Image {i+1}", use_column_width=True)

if __name__ == "__main__":
    main()


Overwriting app.py


In [12]:
!streamlit run app.py &>/dev/null&


In [13]:
!pip install pyngrok


In [14]:
!ngrok authtoken 2cM6tQosgyOMLeO5fgtuaayoTXJ_txjdzgiQnRspg5omXTfw


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
from pyngrok import ngrok
import streamlit as st

# Get the port number where Streamlit is running
port = st.get_option("server.port")

# Setup a tunnel to the streamlit port
public_url = ngrok.connect(port)
public_url


<NgrokTunnel: "https://c59b-35-198-197-171.ngrok-free.app" -> "http://localhost:8501">